### SQL Databases 

In [1]:

## create sample SQLite Database
import sqlite3
import os

os.makedirs("data/databases",exist_ok=True)

In [2]:
## create sample database
conn=sqlite3.connect('data/databases/company.db')
cursor=conn.cursor()

In [3]:
# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
                 (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
                 (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [5]:
# Insert sample data
employees = [
    (1, 'Akarshan Kapoor', 'Data Scientist', 'Analytics', 120000),
    (2, 'Priya Mehta', 'Machine Learning Engineer', 'AI Research', 115000),
    (3, 'Rohan Verma', 'Business Analyst', 'Product Strategy', 90000),
    (4, 'Ananya Sharma', 'Data Engineer', 'Engineering', 102000),
    (5, 'Vikram Singh', 'Senior Data Scientist', 'Advanced Analytics', 135000),
    (6, 'Neha Gupta', 'AI Research Scientist', 'AI Research', 125000),
    (7, 'Siddharth Jain', 'Data Analyst', 'Analytics', 85000),
    (8, 'Isha Kapoor', 'Machine Learning Engineer', 'AI Research', 110000),
    (9, 'Arjun Rao', 'Business Intelligence Analyst', 'Analytics', 95000),
    (10, 'Kavita Nair', 'Senior Data Engineer', 'Engineering', 130000)
]

projects = [
    (1, 'Fraud Detection System', 'Active', 300000, 1),
    (2, 'Customer Churn Prediction', 'Completed', 180000, 2),
    (3, 'Credit Risk Modeling', 'Active', 250000, 5),
    (4, 'Recommendation Engine', 'Planning', 200000, 1),
    (5, 'Sales Forecasting Dashboard', 'Completed', 150000, 3),
    (6, 'RAG Chatbot Implementation', 'Active', 350000, 2),
    (7, 'Time Series Energy Forecast', 'Completed', 220000, 6),
    (8, 'Image Classification Model', 'Active', 300000, 8),
    (9, 'Customer Segmentation Analysis', 'Planning', 180000, 7),
    (10, 'AI-based Maintenance Scheduler', 'Active', 400000, 10)
]


In [6]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [7]:
cursor.execute("SELECT * FROM employees")

In [8]:
conn.commit()
conn.close()


## Database Content Extraction

In [9]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [10]:
## Method 1: SQLDATABASE Utility
print("\n ✅ SQLDatabase Utility")
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL:")
print(db.get_table_info())


 ✅ SQLDatabase Utility
Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	Akarshan Kapoor	Data Scientist	Analytics	120000.0
2	Priya Mehta	Machine Learning Engineer	AI Research	115000.0
3	Rohan Verma	Business Analyst	Product Strategy	90000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	Fraud Detection System	Active	300000.0	1
2	Customer Churn Prediction	Completed	180000.0	2
3	Credit Risk Modeling	Active	250000.0	5
*/


In [11]:
## Method 2: Custom SQL to Document conversion

from typing import List
from langchain_core.documents import Document

In [12]:
print("\n ✅ Custom SQL Processing")

def sql_to_documents(db_path:str)-> List[Document]:
    """Convert SQL Database To documents with context"""
    conn=sqlite3.connect(db_path)
    cursor=conn.cursor()
    documents=[]
    # Strategy 1: Create documents for each table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        
        # Get table schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]
        
        # Get table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        
        # Create table overview document
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"
        
        # Add sample records
        table_content += "Sample Records:\n"
        for row in rows[:5]:  # First 5 records
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
        
        doc = Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )
        documents.append(doc)

     # Strategy 2: Create relationship documents
    # Example: Join employees and projects
    cursor.execute("""
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
    """)
    
    relationships = cursor.fetchall()
    rel_content = "Employee-Project Relationships:\n\n"
    for rel in relationships:
        rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} - Status: {rel[3]}\n"
    
    rel_doc = Document(
        page_content=rel_content,
        metadata={
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee_project_join'
        }
    )
    documents.append(rel_doc)
    
    conn.close()
    return documents


 ✅ Custom SQL Processing


In [13]:
sql_to_documents("data/databases/company.db")

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 10, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 10\n\nSample Records:\n{'id': 1, 'name': 'Akarshan Kapoor', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 120000.0}\n{'id': 2, 'name': 'Priya Mehta', 'role': 'Machine Learning Engineer', 'department': 'AI Research', 'salary': 115000.0}\n{'id': 3, 'name': 'Rohan Verma', 'role': 'Business Analyst', 'department': 'Product Strategy', 'salary': 90000.0}\n{'id': 4, 'name': 'Ananya Sharma', 'role': 'Data Engineer', 'department': 'Engineering', 'salary': 102000.0}\n{'id': 5, 'name': 'Vikram Singh', 'role': 'Senior Data Scientist', 'department': 'Advanced Analytics', 'salary': 135000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 10, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns